In [22]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import requests
import json
import time
import asyncio
from pymongo import MongoClient
from datetime import datetime

In [23]:
def traducir(texto):
    contenido_p = ''
    while True:
        try:
            contenido_p = ts.translate_text(
                texto, translator='bing', to_language='es')
            break
        except Exception as e:
            print(e)
            try:
                contenido_p = ts.translate_text(
                    texto, translator='google', to_language='es')
                break
            except Exception as e:
                print(e)
                pass
            pass
    return contenido_p

In [24]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

In [25]:
url = 'https://novel-bin.net/sort/daily-novel-bin'

In [26]:
driver = uc.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))

In [27]:
driver.get(url)
driver.maximize_window()

In [28]:
time.sleep(5)
list_novels = []
while True:
    try:
        html = driver.page_source
        soup = bs(html, 'html.parser')
        list_novels_page = soup.find_all('h3', class_='novel-title')
        for list in list_novels_page:
            a = list.find('a').get('href')
            list_novels.append(a)
        last = driver.find_element(By.CLASS_NAME, 'last')
        driver.find_element(By.XPATH,'//*[@id="container"]/div[3]/div/ul/li[8]/a').click()
        time.sleep(3)
        break
    except Exception as e:
        print(e)
        break


In [29]:
print(list_novels)

['https://novel-bin.net/novel-bin/return-of-the-shattered-constellation-nov381526030', 'https://novel-bin.net/novel-bin/the-supreme-martial-king-shocking-all-realms-nov5967499', 'https://novel-bin.net/novel-bin/supreme-tamer-nov-927104258', 'https://novel-bin.net/novel-bin/building-a-gaming-empire-from-scratch-nov181733340', 'https://novel-bin.net/novel-bin/this-necromancer-just-wants-to-plant-trees-nov-1081524620', 'https://novel-bin.net/novel-bin/chrysalis-nov399218049', 'https://novel-bin.net/novel-bin/my-lady-to-defeat-scumbags-you-must-start-early-nov-1186385801', 'https://novel-bin.net/novel-bin/hide-and-cultivate-in-the-east-palace-only-to-find-the-prince-is-a-girl-nov-1358746123', 'https://novel-bin.net/novel-bin/master-zhan-calm-down-nov-1265550898', 'https://novel-bin.net/novel-bin/perfect-pampered-marriage-good-morning-hubby-nov-807029362', 'https://novel-bin.net/novel-bin/deities-starting-from-goblins-nov-254673335', 'https://novel-bin.net/novel-bin/please-permit-me-to-love

In [30]:
client = MongoClient('mongodb://localhost:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']

In [49]:
for novel in list_novels:
    coleccion_app_novela = db['app_novela']
    resultado = coleccion_app_novela.find_one({'url': novel})
    print(resultado['_id'] if resultado else 'Novela no registrada')
    driver.get(novel)
    time.sleep(1)
    if not resultado:
        html_novel = driver.page_source
        soup_novel = bs(html_novel, 'html.parser')
        titulo = soup_novel.find('h3', class_='title').getText()
        table_info = soup_novel.find('ul', class_='info info-meta').find_all('li')
        for li in table_info:
            if 'Author:' in li.find('h3').getText().strip().rstrip():
                autor = li.find('a').getText().strip().rstrip()
            if 'Genre:' in li.find('h3').getText().strip().rstrip():
                genero = ', '.join([x.getText().strip().rstrip() for x in li.find_all('a')])
            if 'Status:' in li.find('h3').getText().strip().rstrip():
                status = li.find('a').getText().strip().rstrip()
        img_src = soup_novel.find('div', class_='book').find('img').get('src')
        descripcion = soup_novel.find('div', class_='desc-text').getText().strip().rstrip()
        novela_data ={
            "sitio_id":"65e7e1707963e189aa4c1256",
            "nombre": titulo.upper(),
            "sinopsis": descripcion,
            "autor": autor,
            "genero": genero,
            "status": 'emision' if status =='Ongoing' else 'completo',
            "url": novel,
            "imagen_url": img_src,
            "created_at": datetime.now(),
            "updated_at": datetime.now()
        }
        novela_id = str(coleccion_app_novela.insert_one(novela_data).inserted_id)
    driver.find_element(By.XPATH, '/html/body/div/main/div[2]/div[1]/div[4]/ul/li[2]/a').click()
    time.sleep(4)
    coleccion_app_capitulo = db['app_capitulo']
    html_novel = driver.page_source
    soup_novel = bs(html_novel, 'html.parser')
    panel_chapter = soup_novel.find('div', class_='panel-body').find_all('div',class_='row')
    for row in panel_chapter:
        lists_chapters = row.find_all('ul', class_='list-chapter')
        for ul in lists_chapters:
            a_list = ul.find_all('a')
            for a in a_list:
                resultado_app_capitulo = coleccion_app_capitulo.find_one({'novela_id': str(resultado['_id']), 'url': a.get('href')})
                print(resultado_app_capitulo['_id'] if resultado_app_capitulo else 'capitulo no registrado')
                if not resultado_app_capitulo:
                    capitulo_novela_data = {
                        "novela_id": novela_id,
                        "nombre": str(traducir(a.getText())),
                        "url": str(a.get('href')),
                        "created_at": datetime.now(),
                        "updated_at": datetime.now()
                    }
                    cap_id = str(coleccion_app_capitulo.insert_one(capitulo_novela_data).inserted_id)
    # break


Novela no registrada


TypeError: 'NoneType' object is not subscriptable

In [ ]:
driver.quit()